In [1]:
import urllib
import ntpath
import json
import os
from pipelineapi import PipelineAPI
import time


def get_dataset(ds, data_pipeline_api = None):
    if not data_pipeline_api:
        data_pipeline_api = PipelineAPI("http://ec2-54-184-229-238.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')


    dataset_names = ['pronto_set', 'bdd_set', 'kache_set','china_set']
    os.makedirs(kache_anns_dir, exist_ok = True)

    task, file_name = data_pipeline_api.data_file(video_source_names=[], dataset_names=[ds],
                                categories_names=[], label_statuses=[], bulk_size=6000)
    return task, file_name

dataset_names = ['pronto_set', 'bdd_set', 'kache_set','china_set', 'holdout_set']
data_pipeline_api = PipelineAPI("http://ec2-54-184-229-238.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')
kache_anns_dir =  '/home/kuser/Desktop/all_sets_for_barrier_labels/'
os.makedirs(kache_anns_dir ,exist_ok=True)
for ds in dataset_names:
    
    task, filename = get_dataset(ds, data_pipeline_api)
    print(task); print(filename)

    while not os.path.exists(os.path.join(kache_anns_dir, os.path.splitext(filename)[0]+'_1.json')):
        print(data_pipeline_api.get_file(task_id=task, folder=kache_anns_dir))
        time.sleep(30)

c48d16b3-ef09-4a56-8622-3506089cd0cf
5419f85a-d1b4-4a39-a786-e702c0930764.zip
IN PROCESS
None
SUCCESS
7dcb6481-f871-4852-9e54-b66d5c264180
8dd0e0e3-a3ee-4f44-98ee-84922324dab2.zip
IN PROCESS
None
None
None
None
None
None
None
SUCCESS
e6096fbe-e13f-4b33-9407-5203ba3c6507
46760dc4-244f-468d-acc1-68649ac885e6.zip
None
None
None
None
None
None
SUCCESS
f644e975-fa63-4007-a05e-c305f1e7ce3b
a0b27d33-2b8e-4b0e-a470-dde22828b83a.zip
None
None
SUCCESS
ba708b1c-1990-415e-b3e6-0f1c170b23f8
b8d61b4d-65f7-46bb-9880-21100acb4fbd.zip
IN PROCESS
None
None
SUCCESS


In [4]:
import urllib
import ntpath
import glob


def path_leaf(path):
    if urllib.parse.urlparse(path).scheme != "" or os.path.isabs(path):
        path = os.path.split(path)[-1]
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

In [5]:
for k_file in glob.glob('/home/kuser/Desktop/all_sets_for_barrier_labels/*.json'):
    with open(k_file) as f:
        data = json.load(f)
    for kache_img_data in data:
        kache_img_data['labels'] = []
        
    with open(k_file, 'w') as fn:
        json.dump(data, fn)
        
        

In [8]:
for k_file in glob.glob('/home/kuser/Desktop/all_sets_for_barrier_labels/*.json'):
    with open(k_file) as f:
        data = json.load(f)
        
    scalabel_data_format = []
    total_idx = 0
    key_mappings  = {} #kache_id-> scalabel_id
    # Get Pending Precision Labels
    
    for frame in data:
        img_name = path_leaf(frame['name'])    

        scalabel_frame = {
            'url': frame['url'],
            'name': frame['name'],
            'width': frame['width'],
            'height': frame['height'],
            'labels': [],
            'index': total_idx,
            'timestamp': frame['timestamp'],
            'videoName': '',
            'attributes': {}
        }
        frame['scalabel_id'] = total_idx
        frame['index'] = total_idx
        scalabel_data_format.append(scalabel_frame)
        total_idx += 1
        
    with open(k_file, 'w+') as fn:
        json.dump(data, fn)
        
    with open(os.path.join('/home/kuser/Desktop/all_sets_for_barrier_labels/scalabel/', path_leaf(k_file)), 'w+') as fn:
        json.dump(scalabel_data_format, fn)


In [12]:
from itertools import zip_longest
import urllib.parse as urlparse
from urllib.parse import urlencode
    
def push_to_scalabel(scalabel_file):
    with open("/media/kuser/datastore/nebula/pipelines/configs/entanglement_cfg.tml", 'r') as tmlfile:
        cfg = toml.load(tmlfile)


    args = cfg['scalabel']['args']
    scalabel_api_uri = "{}postProject?{}".format(
        cfg['scalabel']['base_url'], urlencode(args))

    project_name = os.path.splitext(path_leaf(scalabel_file))[0]


    body = {"project_name": project_name,
            "item_type": cfg['scalabel']['item_type'],
            "label_type": cfg['scalabel']['label_type'],
            "page_title": "Label Missing Barriers",
            "task_size": "6000",
            "vendor_id": str(cfg['scalabel']['vendor_id']),
            }

    files = {'item_file': open(scalabel_file, 'rb'),
             'categories': open(cfg['scalabel']['categories_file'], 'rb'),
             'attributes': open(cfg['scalabel']['attributes_file'], 'rb')}

    try:
        print('Initializing Project: {}'.format(project_name))
        resp = requests.post(scalabel_api_uri, files=files, data=body)
    except requests.exceptions.Timeout as e1:
        # Maybe set up for a retry, or continue in a retry loop
        print(e1)
    except requests.exceptions.TooManyRedirects as e2:
        # Tell the user their URL was bad and try a different one
        print(e2)
    except requests.exceptions.RequestException as e3:
        # catastrophic error. bail.
        print(e3)
        sys.exit(1)

    if resp.status_code != 200 or resp.content != ''.encode('UTF-8'):
        raise requests.exceptions.RequestException(
            'POST /create status code: {}, content {}'.format(resp.status_code, resp.content))

    if body['label_type'] == 'box2d' or body['label_type'] == 'lane':
        lbl_type = 'label2d'

    active_precision_uri = f"{cfg['scalabel']['base_url']}{lbl_type}?project_name={project_name}&task_index=0"
    print('Successfully Created Project. URL:', active_precision_uri)
    return active_precision_uri

In [14]:
import toml
import argparse
import requests

for k_file in glob.glob('/home/kuser/Desktop/all_sets_for_barrier_labels/scalabel/*.json'):
    push_to_scalabel(k_file)

Initializing Project: 8dd0e0e3-a3ee-4f44-98ee-84922324dab2_10
Successfully Created Project. URL: http://ec2-34-220-1-2.us-west-2.compute.amazonaws.com:8686/label2d?project_name=8dd0e0e3-a3ee-4f44-98ee-84922324dab2_10&task_index=0
Initializing Project: b8d61b4d-65f7-46bb-9880-21100acb4fbd_3
Successfully Created Project. URL: http://ec2-34-220-1-2.us-west-2.compute.amazonaws.com:8686/label2d?project_name=b8d61b4d-65f7-46bb-9880-21100acb4fbd_3&task_index=0
Initializing Project: b8d61b4d-65f7-46bb-9880-21100acb4fbd_4
Successfully Created Project. URL: http://ec2-34-220-1-2.us-west-2.compute.amazonaws.com:8686/label2d?project_name=b8d61b4d-65f7-46bb-9880-21100acb4fbd_4&task_index=0
Initializing Project: a0b27d33-2b8e-4b0e-a470-dde22828b83a_1
Successfully Created Project. URL: http://ec2-34-220-1-2.us-west-2.compute.amazonaws.com:8686/label2d?project_name=a0b27d33-2b8e-4b0e-a470-dde22828b83a_1&task_index=0
Initializing Project: 8dd0e0e3-a3ee-4f44-98ee-84922324dab2_22
Successfully Created Pro

Successfully Created Project. URL: http://ec2-34-220-1-2.us-west-2.compute.amazonaws.com:8686/label2d?project_name=a0b27d33-2b8e-4b0e-a470-dde22828b83a_3&task_index=0
Initializing Project: 5419f85a-d1b4-4a39-a786-e702c0930764_11
Successfully Created Project. URL: http://ec2-34-220-1-2.us-west-2.compute.amazonaws.com:8686/label2d?project_name=5419f85a-d1b4-4a39-a786-e702c0930764_11&task_index=0
Initializing Project: 8dd0e0e3-a3ee-4f44-98ee-84922324dab2_14
Successfully Created Project. URL: http://ec2-34-220-1-2.us-west-2.compute.amazonaws.com:8686/label2d?project_name=8dd0e0e3-a3ee-4f44-98ee-84922324dab2_14&task_index=0
Initializing Project: 46760dc4-244f-468d-acc1-68649ac885e6_5
Successfully Created Project. URL: http://ec2-34-220-1-2.us-west-2.compute.amazonaws.com:8686/label2d?project_name=46760dc4-244f-468d-acc1-68649ac885e6_5&task_index=0
Initializing Project: 5419f85a-d1b4-4a39-a786-e702c0930764_7
Successfully Created Project. URL: http://ec2-34-220-1-2.us-west-2.compute.amazonaws

In [ ]:
from pipelineapi import PipelineAPI
import os

def dataset_import(dataset_name, dataset_file):
        data_pipeline_api = PipelineAPI("http://ec2-54-149-236-75.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')
        with open(dataset_file, 'rb') as f:
            data_pipeline_api.upload_json_file(f, file_name=path_leaf(dataset_file), round='inspection', dataset_name=dataset_name, skip_image=True)
            

In [ ]:
from pipelineapi import PipelineAPI
import os

def dataset_import(dataset_name, dataset_file):
        data_pipeline_api = PipelineAPI("http://ec2-54-191-65-138.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')
        with open(dataset_file, 'rb') as f:
            data_pipeline_api.upload_json_file(f, file_name=path_leaf(dataset_file), round='inspection', dataset_name=dataset_name, skip_image=False)
            

In [131]:
import urllib
import ntpath
import json
import os
from pipelineapi import PipelineAPI

def get_dataset():
    data_pipeline_api = PipelineAPI("http://ec2-54-184-229-238.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')

    batch_size = 0
    batch = 0
    batch_anns = []


    dataset_names = ['holdout_set']
    kache_anns_dir =  '/home/dean/Desktop/pronto_set/'
    os.makedirs(kache_anns_dir, exist_ok = True)

    databunch = data_pipeline_api.data(dataset_names=dataset_names, video_source_names=[], categories_names=[],
                    label_statuses=[42], page_size=None, frame_id=None) # returns generator

    for anns in databunch:
#           anns = update_traffic_light_cats([anns])
        batch_anns.append(anns)
        batch_size += len(anns)
        if batch_size % 250001 == 0:
            with open(os.path.join(kache_anns_dir,f'{dataset_names[0]}--accuracy-rnd.json'), 'w') as fn:
                json.dump(batch_anns, fn)
            batch_anns = []
            batch += 1

    # Save any remaining frames
    if len(batch_anns) != 0:
        with open(os.path.join(kache_anns_dir,f'{dataset_names[0]}--accuracy-rnd.json'), 'w') as fn:
            json.dump(batch_anns, fn)


In [132]:
get_dataset()

In [126]:
import urllib
import ntpath
from pipelineapi import PipelineAPI
import os
import numpy as np
import json
import glob


def path_leaf(path):
    if urllib.parse.urlparse(path).scheme != "" or os.path.isabs(path):
        path = os.path.split(path)[-1]
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

In [ ]:
1562750429.150900

In [ ]:
for p_file in glob.glob('/media/dean/datastore/nebula/pipelines/kache/pulsar/precision/active_precision_projects/*.json'):
    with open(p_file) as f:
        precision_data = json.load(f)
    for kache_id, kache_img_data in precision_data.items():
        for lbl in kache_img_data['labels']:
            print(f"File: {p_file} Scalabel Label Id {lbl['scalabel_label_id']} kache label id: {lbl['kache_label_id']}" )

In [133]:
from collections import defaultdict


hfile = '/home/dean/Desktop/pronto_set/holdout_set--accuracy-rnd.json'
with open(hfile) as f:
    precision_data3 = json.load(f)
    
precision_data =precision_data3


urls = defaultdict(list)
for sfile in glob.glob('/home/dean/Desktop/pronto_set/scalabel_files/*.json'):
    with open(sfile) as f:
        data = json.load(f)
                 
    for img_data in data["Task"]["items"]:
        found= False
        for kache_img_data in precision_data:
            if kache_img_data['url'] == img_data['url']:
                kache_img_data['scalabel_id'] = img_data['index']
                kache_img_data['index'] = img_data['index']
                
                # Match labels
                if img_data.get("labelIds"):
                    for lbl_idx in sorted(img_data["labelIds"]):
                        scalabel_lbl = [lb for lb in data["Labels"] if lb['id'] == lbl_idx][0]
                        # Find kache label
                        if kache_img_data.get("labels"):
                            for lbl in kache_img_data['labels']:
                                scanns = np.array([scalabel_lbl['data']['x'], scalabel_lbl['data']['y'], scalabel_lbl['data']['x'] + scalabel_lbl['data']['w'] -0.5 , scalabel_lbl['data']['y'] + scalabel_lbl['data']['h'] -0.5])
                                if lbl.get('box2d',None):
                                    kanns = np.array([lbl['box2d']['x1'], lbl['box2d']['y1'], lbl['box2d']['x2'], lbl['box2d']['y2']])
                                elif lbl.get('category',None) in ['lane', 'drivable area']:
                                    lbl['label_status'] = 31
                                    continue
                                else:
                                    kanns = np.array([0,0,0,0])
                                        
                                if np.allclose(scanns, kanns, atol=6.5):
                                    print(f"matching {scalabel_lbl['id']} with {lbl['kache_label_id']}")
                                    lbl['scalabel_label_id'] = scalabel_lbl['id']
                                
                
                
                
                urls[path_leaf(sfile)].append(kache_img_data)
                found = True
                break
                    
            if found: break
    fname = os.path.splitext(path_leaf(sfile))[0]
    with open(f"/home/dean/Desktop/kache_files/{fname}.json", "w+") as f:
        json.dump(list(urls[path_leaf(sfile)]), f)     
                 

matching 0 with 3021857
matching 1 with 3021858
matching 2 with 3021860
matching 3 with 3021861
matching 4 with 3021862
matching 5 with 3021863
matching 6 with 3021864
matching 7 with 3021859
matching 8 with 3022160
matching 9 with 3022161
matching 10 with 3022162
matching 11 with 3022164
matching 12 with 3022165
matching 13 with 3022166
matching 14 with 3022167
matching 15 with 3022168
matching 16 with 3022169
matching 17 with 3022163
matching 18 with 3022874
matching 19 with 3022876
matching 20 with 3022877
matching 21 with 3022875
matching 22 with 3022882
matching 23 with 3022884
matching 24 with 3022885
matching 25 with 3022886
matching 26 with 3022887
matching 27 with 3022888
matching 28 with 3022889
matching 29 with 3022890
matching 30 with 3022891
matching 31 with 3022883
matching 32 with 3022892
matching 33 with 3022893
matching 34 with 3022894
matching 35 with 3023086
matching 36 with 3023087
matching 37 with 3023088
matching 38 with 3023089
matching 39 with 3023090
matching 4

In [ ]:
import numpy as np


def found_deleted_label(kache_lbl_idx, deleted_lbls):
    found = False
    del_lbl = None
    for lbl_idx, lbl in deleted_lbls.items():
        if kache_lbl_idx.get('scalabel_label_id', None):
            if kache_lbl_idx['scalabel_label_id'] == lbl_idx:
                found = True
                del_lbl = lbl
                return found, del_lbl
    
    return found, del_lbl

In [ ]:
import numpy as np


def update_kache_label(kache_lbl_idx, found_lbls):
    for lbl_idx, lb_dict in found_lbls.items():
        scalabel_lbl = lb_dict['label']
        if kache_lbl_idx.get('scalabel_label_id', None):
            if kache_lbl_idx['scalabel_label_id'] == lbl_idx:
                kache_lbl_idx['category'] = scalabel_lbl['categoryPath']
                kache_lbl_idx['label_status'] = 30
                if not kache_lbl_idx.get('box2d', None):
                    kache_lbl_idx['box2d'] = {'x1': scalabel_lbl['data']['x'],
                                              'y1': scalabel_lbl['data']['y'],
                                              'x2': scalabel_lbl['data']['x'] + scalabel_lbl['data']['w'] -0.5,
                                              'y2': scalabel_lbl['data']['y'] + scalabel_lbl['data']['h'] -0.5,}
                else:  
                    kache_lbl_idx['box2d']['x1'] = scalabel_lbl['data']['x']
                    kache_lbl_idx['box2d']['y1'] = scalabel_lbl['data']['y']
                    kache_lbl_idx['box2d']['x2'] = scalabel_lbl['data']['x'] + scalabel_lbl['data']['w'] -0.5
                    kache_lbl_idx['box2d']['y2'] = scalabel_lbl['data']['y'] + scalabel_lbl['data']['h'] -0.5

                if scalabel_lbl.get('attributes', None):
                    kache_lbl_idx['attributes']['occluded'] = scalabel_lbl['attributes'].get('occluded', False) if scalabel_lbl['attributes'].get('occluded', False) else scalabel_lbl['attributes'].get('Occluded', False)
                    kache_lbl_idx['attributes']['truncated'] = scalabel_lbl['attributes'].get('truncated', False) if scalabel_lbl['attributes'].get('truncated', False) else scalabel_lbl['attributes'].get('Truncated', False)
                    kache_lbl_idx['attributes']['trafficLightColor'] = scalabel_lbl['attributes'].get('trafficLightColor', [0, 'NA']) if scalabel_lbl['attributes'].get('trafficLightColor', False) else scalabel_lbl['attributes'].get('Traffic Light Color', False)
                    kache_lbl_idx['attributes']['semi'] = scalabel_lbl['attributes'].get('semi', False) if scalabel_lbl['attributes'].get('semi', False) else scalabel_lbl['attributes'].get('Semi', False)
                    kache_lbl_idx['attributes']['low_visibility'] = scalabel_lbl['attributes'].get('low_visibility', False) if scalabel_lbl['attributes'].get('low_visibility', False) else scalabel_lbl['attributes'].get('Low_Visibility', False)


In [ ]:
import glob
import json
import copy
import os

top_indices = {}

# Open original scalabel file, get largest id
for sca_file in glob.glob('/home/dean/Desktop/pronto_set/scalabel_files/*.json'):
    with open(sca_file) as f:
        scalabel_data = json.load(f)
    if scalabel_data.get("Labels", None):
        original_labels = copy.deepcopy(scalabel_data["Labels"])
        top_indices[path_leaf(sca_file)] = scalabel_data["Labels"][-1]['id']
        print(f"Largest {path_leaf(sca_file)} index:", top_indices[path_leaf(sca_file)] )
        
        


# Open latest corresponding file Scalabel File
for sca_file in glob.glob('/home/dean/Desktop/pronto_set/scalabel_files/results/*.json'):
    found_lbls = {}
    created_lbls = {}
    deleted_lbls = {}
    sca_key = path_leaf(sca_file.replace('_results', ''))
    with open(sca_file) as f:
        scalabel_data = json.load(f)
        
    tot_idx = 0
    # Loop through each label and each index.
    for img_data in scalabel_data["Task"]["items"]:
        if img_data.get("labelIds"):
            for lbl_idx in sorted(img_data["labelIds"]):
                #print(lbl_idx)
                scalabel_label = [lbl for lbl in scalabel_data["Labels"] if lbl['id'] == lbl_idx][0]

                if lbl_idx == tot_idx: # Found match
                    found_lbls[lbl_idx]= { 'url': img_data['url'], 'frame_idx': img_data['index'], 'label': copy.deepcopy(scalabel_label) }
                # If the number is greater than the largest index, push to created stack
                elif lbl_idx > top_indices[sca_key]: # Created label
                    created_lbls[lbl_idx] = { 'url': img_data['url'], 'frame_idx': img_data['index'], 'label': copy.deepcopy(scalabel_label) }
                # If the number is less than largest index but more than 1 from previous index, record missing index as deleted
                else: # Deleted labels discovered
                    #print(f'Missing labels: lbl_idx:{lbl_idx}, tot_idx:{tot_idx}')
                    # Add current label to found
                    found_lbls[lbl_idx]= { 'url': img_data['url'], 'frame_idx': img_data['index'], 'label': copy.deepcopy(scalabel_label) }
                    while tot_idx < lbl_idx:
                        #print(f'Appending label to delete lbl_idx:{tot_idx}')
                        deleted_lbls[tot_idx]= { 'url': img_data['url'], 'frame_idx': img_data['index'], 'label': copy.deepcopy(scalabel_label) }
                        tot_idx += 1
                    

                tot_idx += 1
                
                
    for i in range(top_indices[sca_key]):
        if i not in found_lbls.keys() and i not in created_lbls.keys() and i not in deleted_lbls.keys():
            del_lbl = [lbl for lbl in original_labels if lbl['id'] == i]
            if del_lbl:
                deleted_lbls[i] = { 'url': None, 'frame_idx': None, 'label': copy.deepcopy(del_lbl[0]) }
            
    print(f"http://ec2-34-220-1-2.us-west-2.compute.amazonaws.com:8686/label2d?project_name={path_leaf(sca_file.replace('_results.json',''))}&task_index=0")
    print("total Altered Labels", len(set(found_lbls)))
    print("total Created Labels", len(set(created_lbls)))
    print("total Deleted Labels", len(set(deleted_lbls)))
    print("Total Labels for Payment Processing: ", len(found_lbls) + len(deleted_lbls))
    
    ## Update Kache Labels ##
    kache_file = f"/home/dean/Desktop/kache_files/{sca_key}"
    print(kache_file)
    if os.path.exists(kache_file):
        with open(kache_file) as f:
            kache_data = json.load(f)
        tot_del, tot_match = 0, 0
        
        for img_data in kache_data:
            if img_data.get("labels", None):
                for kache_lbl_idx in img_data["labels"]:
                    # Find corresponding labels
                    fnd, lbl = found_deleted_label(kache_lbl_idx, deleted_lbls)
                    if fnd:
                        tot_del +=1
                        kache_lbl_idx['label_status'] = 31
                        #print(f"Found deleted label {kache_lbl_idx['scalabel_label_id']} for kache label {kache_lbl_idx['kache_label_id']}")
                    elif kache_lbl_idx['scalabel_label_id'] in found_lbls.keys():
                        update_kache_label(kache_lbl_idx, found_lbls)
                        #print(f"Found matching label {kache_lbl_idx['scalabel_label_id']} for kache label {kache_lbl_idx['kache_label_id']}")
                        tot_match +=1
                    elif kache_lbl_idx['scalabel_label_id'] in created_lbls.keys(): 
                        print(f"Found created label {kache_lbl_idx['scalabel_label_id']} for kache label {kache_lbl_idx['kache_label_id']}")
        print(f"Total match: {tot_match}, Total deleted: {tot_del}")
                              
    
    fname = os.path.splitext(path_leaf(sca_file))[0]
    with open(f"/home/dean/Desktop/complete_kache_files/{fname}.json", "w+") as f:
        json.dump(list(kache_data), f)



In [ ]:
from pipelineapi import PipelineAPI
import os

def dataset_import(dataset_name, dataset_file):
        data_pipeline_api = PipelineAPI("http://ec2-54-184-229-238.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')
        with open(dataset_file, 'rb') as f:
            data_pipeline_api.upload_json_file(f, file_name=path_leaf(dataset_file), round='precision', dataset_name=dataset_name, skip_image=False)
            

In [130]:
import datetime
import copy

precision_dir = '/home/dean/Desktop/complete_kache_files/precision/'
os.makedirs(precision_dir, exist_ok = True)

for file in glob.glob('/media/dean/datastore/nebula/pipelines/kache/redshift/precision/labeled_precision_projects/*.json'):
    with open(file) as f:
        precision_data = json.load(f)
    kachedb_datasets = {}
    for kachedb_frame_dict in precision_data:

       
        for lbl in kachedb_frame_dict['labels']:
            if lbl['label_status'] == 32:
                lbl['label_status'] = 30
        
        if kachedb_frame_dict['dataset_name'] in kachedb_datasets.keys():
            kachedb_datasets[kachedb_frame_dict['dataset_name']].append(copy.deepcopy(kachedb_frame_dict))
        else:
            kachedb_datasets[kachedb_frame_dict['dataset_name']] = [copy.deepcopy(kachedb_frame_dict)]

    for dataset_name, frames in kachedb_datasets.items():
        ## Save Labeled Precision Json Files ##
        d = datetime.datetime.utcnow()
        epoch = datetime.datetime(1970,1,1)
        ts = int((d - epoch).total_seconds())
        labeled_kachedb_annotations_file = os.path.join(
            precision_dir, f"labeled_{dataset_name}--{ts}.json")
        with open(f"{labeled_kachedb_annotations_file}", 'w+') as fn:
            json.dump(frames, fn)
            
        dataset_import(dataset_name, labeled_kachedb_annotations_file)

In [ ]:
dataset_import("bdd_set", '/home/dean/Desktop/complete_kache_files/precision/labeled_bdd_set--1564199177.json')

In [ ]:
import glob
import json
import os

os.makedirs('/home/dean/Desktop/complete_kache_files/', exist_ok = True)
for kfile in glob.glob('/home/dean/Desktop/kache_files/*.json'):
    with open(kfile) as f:
        kache_data = json.load(f)
    # Get corresponding scalabel urls
    project_name = path_leaf(kfile.replace('_kache_data.json', ''))
    # Find corresponding scalabel label id from scalabel files
    with open(f'/home/dean/Desktop/pronto_set/scalabel_files/{project_name}.json') as f:
        scalabel_data = json.load(f)
    
    for url, kache_img_data in kache_data.items():        
        scalabel_lbl_maches = {}
        for scalabel_img_data in scalabel_data["Task"]["items"]:
            if scalabel_img_data['url'] == url: # found corresponding frame
                #print(f"Found correspoding frame: {project_name}|{url}|{kache_img_data['kache_id']}")
                kache_img_data['scalabel_id'] = scalabel_img_data['index']
                kache_img_data['index'] = scalabel_img_data['index']
                
                # Update scalabel_label_ids for each kache_label_id
                if kache_img_data.get('labels', None):
                    for kache_idx, kache_label in enumerate(kache_img_data['labels']):
                        found = False
                      
                          
                        if scalabel_img_data.get('labelIds', None):
                      
                            for scalabel_idx in scalabel_img_data['labelIds']:
                                scalabel_label = [lbl for lbl in scalabel_data["Labels"] if lbl['id'] == scalabel_idx][0]
                                
                                if kache_label.get('box2d', None) and \
                                    kache_label['box2d']['x1'] == scalabel_label['data']['x']  and \
                                    kache_label['box2d']['y1'] == scalabel_label['data']['y']:
#                                     print('Found match for kache label:', kache_label['kache_label_id'])
#                                     print("KACHE DATA:", kache_label)
#                                     print("SCALABEL DATA:", scalabel_label)

                                    # Found matching label 
                                    found = True
                                    kache_label['scalabel_label_id'] = scalabel_label['id']
                                    scalabel_lbl_maches[kache_label['kache_label_id']] = scalabel_label['id']
                                    break
                                
                    # Assign nonmatches to crresponding scalabel label with matching category
                    all_lbls = [lbl['kache_label_id'] for lbl in kache_img_data['labels']]
                    if list(scalabel_lbl_maches.keys()):
                        print('MATCHES:', list(scalabel_lbl_maches.keys()))
                    nonmatches = set(all_lbls) - set(list(scalabel_lbl_maches.keys()))
                    #print('NONMATCHES:', nonmatches)
                    
                    for kache_idx, kache_label in enumerate(kache_img_data['labels']):
                        if kache_label['kache_label_id'] in nonmatches:
                            if scalabel_img_data.get('labelIds', None):
                      
                                for scalabel_idx in scalabel_img_data['labelIds']:
                                    scalabel_label = [lbl for lbl in scalabel_data["Labels"] if lbl['id'] == scalabel_idx][0]


                                    # Pair remaining labels with corresponding category
                                    if scalabel_label['id'] not in scalabel_lbl_maches.values():

                                        if kache_label['category'] == scalabel_label['categoryPath']:
                                            # Sync ids for importing changes later
                                            kache_label['scalabel_label_id'] = scalabel_label['id']
                                            scalabel_lbl_maches[kache_label['kache_label_id']] = scalabel_label['id']
                                            # Update nonmatches
                                            nonmatches = set(all_lbls) - set(list(scalabel_lbl_maches.keys()))
                                            break
    
    with open(f"/home/dean/Desktop/complete_kache_files/{project_name}.json", "w+") as f:
        json.dump(list(kache_data.values()), f)           

In [ ]:
# import glob
# import json
# import os

# os.makedirs('/home/dean/Desktop/complete_kache_files/', exist_ok = True)
# for kfile in glob.glob('/home/dean/Desktop/kache_files/*.json'):
#     with open(kfile) as f:
#         kache_data = json.load(f)
#     # Get corresponding scalabel urls
#     project_name = path_leaf(kfile.replace('_kache_data.json', ''))
#     # Find corresponding scalabel label id from scalabel files
#     with open(f'/home/dean/Desktop/pronto_set/scalabel_files/{project_name}.json') as f:
#         scalabel_data = json.load(f)
    
#     for url, kache_img_data in kache_data.items():        
#         scalabel_lbl_maches = {}
#         for scalabel_img_data in scalabel_data["Task"]["items"]:
#             if scalabel_img_data['url'] == url: # found corresponding frame
#                 print(f"Found correspoding frame: {project_name}|{url}|{kache_img_data['kache_id']}")
#                 kache_img_data['scalabel_id'] = scalabel_img_data['index']
#                 kache_img_data['index'] = scalabel_img_data['index']
                
#                 # Update scalabel_label_ids for each kache_label_id
#                 if kache_img_data.get('labels', None):
#                     for kache_idx, kache_label in enumerate(kache_img_data['labels']):
#                         found = False
                      
                          
#                         if scalabel_img_data.get('labels', None):
#                             for scalabel_idx, scalabel_label in enumerate(scalabel_img_data['labels']):
#                                 if kache_label.get('box2d', None) and \
#                                     kache_label['box2d']['x1'] == scalabel_label['box2d']['x1']  and \
#                                     kache_label['box2d']['y1'] == scalabel_label['box2d']['y1']  and \
#                                     kache_label['box2d']['x2'] == scalabel_label['box2d']['x2']  and \
#                                     kache_label['box2d']['y2'] == scalabel_label['box2d']['y2']:

#                                     # Found matching label 
#                                     found = True
#                                     kache_label['scalabel_label_id'] = scalabel_label['id']
#                                     scalabel_lbl_maches[kache_label['kache_label_id']] = scalabel_label['id']
#                                     break
                                
#                     # Assign nonmatches to crresponding scalabel label with matching category
#                     all_lbls = [lbl['kache_label_id'] for lbl in kache_img_data['labels']]
#                     print('MATCHES:', list(scalabel_lbl_maches.keys()))
#                     nonmatches = set(all_lbls) - set(list(scalabel_lbl_maches.keys()))
#                     print('NONMATCHES:', nonmatches)
                    
#                     for kache_idx, kache_label in enumerate(kache_img_data['labels']):
#                         if kache_label['kache_label_id'] in nonmatches:
#                             if scalabel_img_data.get('labels', None):
#                                 for scalabel_idx, scalabel_label in enumerate(scalabel_img_data['labels']):
#                                     # Pair remaining labels with corresponding category
#                                     if scalabel_label['id'] not in scalabel_lbl_maches.values():

#                                         if kache_label['category'] == scalabel_label['category']:
#                                             # Sync ids for importing changes later
#                                             kache_label['scalabel_label_id'] = scalabel_label['id']
#                                             scalabel_lbl_maches[kache_label['kache_label_id']] = scalabel_label['id']
#                                             # Update nonmatches
#                                             nonmatches = set(all_lbls) - set(list(scalabel_lbl_maches.keys()))
#                                             break
    
#     with open(f"/home/dean/Desktop/complete_kache_files/{project_name}.json", "w+") as f:
#         json.dump(list(kache_data.values()), f)           

In [ ]:
import urllib
import ntpath
import json
import os
from pipelineapi import PipelineAPI
import datetime

def get_dataset():
    d = datetime.datetime.utcnow()
    epoch = datetime.datetime(1970,1,1)
    timestamp = int((d - epoch).total_seconds())
    batch_size = 0
    batch = 0
    batch_anns = []
    data_pipeline_api = PipelineAPI("http://ec2-54-184-229-238.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')


    dataset_names = ['holdout_set', 'pronto_set', 'bdd_set']
    kache_anns_dir =  '/media/dean/datastore/label_task_data/'
    
    for dataset_name in dataset_names:

        databunch = data_pipeline_api.data(dataset_names=dataset_name, video_source_names=[], categories_names=[],
                        label_statuses=[20], page_size=None, frame_id=None) # returns generator

        for anns in databunch:
            batch_anns.append(anns)
            batch_size += len(anns)
            if batch_size % 10000 == 0:
                with open(os.path.join(kache_anns_dir,'precision--{}--{}--{}.json'.format(dataset_name, batch, timestamp)), 'w') as fn:
                    json.dump(batch_anns, fn)
                batch_anns = []
                batch += 1

        # Save any remaining frames
        if len(batch_anns) != 0:
            with open(os.path.join(kache_anns_dir,'precision--{}--{}--{}.json'.format(dataset_name, batch, timestamp)), 'w') as fn:
                json.dump(batch_anns, fn)


In [ ]:
import glob

all_files = [
    '/media/dean/datastore/datasets/071519_flickers/*.json'
]



for file in glob.glob('/home/dean/Desktop/071519_flickers/*.json'):
    
    

    dataset_name = 'pronto_set'
    print(f'Importing file {file}  to db: {dataset_name}')
    
    

    dataset_import(dataset_name, file)

In [ ]:
def get_dataset():
    data_pipeline_api = PipelineAPI("http://ec2-54-191-65-138.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')

    batch_size = 0
    batch = 0
    batch_anns = []


    dataset_names = 'china_set'
    kache_anns_dir =  '/home/dean/Desktop'

    databunch = data_pipeline_api.data(dataset_names=dataset_names, video_source_names=[], categories_names=[],
                    label_statuses=[20], page_size=None, frame_id=None) # returns generator

    for anns in databunch:
#             anns = update_traffic_light_cats([anns])
        batch_anns.append(anns)
        batch_size += len(anns)
        if batch_size % 5000 == 0:
            with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--rejected.json'), 'w') as fn:
                json.dump(batch_anns, fn)
            batch_anns = []
            batch += 1

    # Save any remaining frames
    if len(batch_anns) != 0:
        with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--rejected.json'), 'w') as fn:
            json.dump(batch_anns, fn)


In [ ]:
get_dataset()

In [ ]:
manifast1 = '/media/dean/datastore/nebula/pipelines/kache/models/AQEBFEkYOQOfd7FIg9Z5/data/coco/labels/manifast_1.txt'
manifast2 = '/media/dean/datastore/nebula/pipelines/kache/models/AQEBFEkYOQOfd7FIg9Z5/data/coco/labels/manifast_2.txt'


In [ ]:
import glob

manifasts = glob.glob(os.path.join('/media/dean/datastore/nebula/pipelines/kache/models/AQEB9tV4ZWVzIz9teLfN/data/coco/labels/train2019/', 'manifast*'))

f = open(os.path.join('/media/dean/datastore/nebula/pipelines/kache/models/AQEB9tV4ZWVzIz9teLfN/data/coco/labels/train2019/',"manifasts.txt"), "w")
for manifast in manifasts:
    mf = open(manifast, "r")
    f.write(mf.read())